<a href="https://colab.research.google.com/github/YuehMintTai/GAETest20210722/blob/main/%E4%B8%89%E7%B8%BD%E5%8C%97%E6%8A%95%E9%96%80%E8%A8%BA%E5%8A%A9%E6%89%8B3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##三總北投門診助手 3.0  2021/8/22

In [28]:
!pip install flask_ngrok
!pip install line-bot-sdk
!pip install bs4
!pip install requests

In [29]:
import bs4
bs4.__version__
import requests
requests.__version__
!pip list

Package                       Version
----------------------------- --------------
absl-py                       0.12.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.1.0
appdirs                       1.4.4
argcomplete                   1.12.3
argon2-cffi                   20.1.0
arviz                         0.11.2
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.2.0
audioread                     2.1.9
autograd                      1.3
Babel                         2.9.1
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        4.0.0
blis                          0.4.1
bokeh                         2.3.3
Bottleneck                    1.3.2
branca                        0.4.2
bs4                           0.0.1
CacheControl

In [40]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, abort
from datetime import date
from linebot import (LineBotApi, WebhookHandler)
from linebot.exceptions import (InvalidSignatureError)
from linebot.models import (MessageEvent, TextMessage, TextSendMessage)
from bs4 import BeautifulSoup
import requests, json
import OPDAid3_SN
today=date.today()
app=Flask(__name__)

line_bot_api=LineBotApi(OPDAid3_SN.myLineBotApi)
handler=WebhookHandler(OPDAid3_SN.myWebhookHandler)

###DrList
target_url='https://wwwv.tsgh.ndmctsgh.edu.tw/Doclist/194/30000/25075'
responses=requests.get(target_url)
soup=BeautifulSoup(responses.text,"html.parser")
myDivs=soup.find_all('div',{'class':'column column-block lg-txt'})
DrList=''
for myDiv in myDivs:
  myLink=myDiv.find('a')
  DrList += str(myLink.contents[0])
  DrList += '\n'
  DrList += f"https://wwwv.tsgh.ndmctsgh.edu.tw/{myLink.get('href')}"
  DrList += '\n\n' 

###OPDList
target_url='https://wwwv.tsgh.ndmctsgh.edu.tw/unit/30000/28387'
responses=requests.get(target_url)
soup=BeautifulSoup(responses.text,"html.parser")
myDiv=soup.select_one('div.systems-information')
myList=myDiv.find_all('a')
OPDList=''
for tag in myList:
  OPDList=str(tag.get('href'))
  OPDList=f"https://wwwv.tsgh.ndmctsgh.edu.tw/{myLink}"
  OPDList +=tag.contents[0]
  OPDList +='\n'
  OPDList += myLink
  OPDList +='\n'

###FeeList
target_url='https://wwwv.tsgh.ndmctsgh.edu.tw/unit/30000/16048'
response=requests.get(target_url)
soup=BeautifulSoup(response.text, 'html.parser')
myDiv=soup.select_one('div.systems-information')
FeeLists=myDiv.find_all('a')
FeeList=''
for i, tag in enumerate(FeeLists):
  FeeList+='\n {}.{}。詳情如下: \n {}'.format(i,tag.contents[0],tag.get('href'))

###NewsList
target_url='https://wwwv.tsgh.ndmctsgh.edu.tw/news/194/30000/24181'
responses=requests.get(target_url)
soup=BeautifulSoup(responses.text,"html.parser")
myNews=soup.find_all('a',{'id':'clickElement'})
NewsList=''
for i, news in enumerate(myNews):
  NewsList +='第{}則.{}...'.format(i,str(news.contents[0]).replace('\n',''))
  NewsList +='\n'
  NewList +='https://wwwv.tsgh.ndmctsgh.edu.tw'+news.get('href')
  NewList +='\n'

run_with_ngrok(app)

@app.route('/')
def index():
  return '三總北投門診助手3.0 '+str(today)

@app.route('/callback',methods=['POST'])
def callback():
  signature=request.headers['X-Line-Signature']
  body=request.get_data(as_text=True)
  app.logger.info("Request body:"+ body)
  try:
    handler.handle(body,signature)
  except InvalidSignatureError:
    abort(400)
  return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
  from bs4 import BeautifulSoup
  import requests
  msg=event.message.text
  if msg=='醫師查詢':
    line_bot_api.reply_message(event.reply_token,TextSendMessage(text=DrList))
  if msg=='門診時間':
    line_bot_api.reply_message(event.reply_token,TextSendMessage(text=OPDList))
  if msg=='掛號查詢': 
    myResult='''
新民路急診部現場掛號：
  台北市北投區新民路60號
中和街門診部現場掛號：
  台北市北投區中和街250號
網路掛號：http://818h.mnd.gov.tw/Tre/start.asp 
人工掛號：02-28932743 或 02-28935869
    '''
    line_bot_api.reply_message(event.reply_token,TextSendMessage(text=myResult))

  if msg=='費用申請':

    line_bot_api.reply_message(event.reply_token,TextSendMessage(text=FeeList))

  if msg=='重大公告':

    line_bot_api.reply_message(event.reply_token,TextSendMessage(text=NewsList))

  else:
    line_bot_api.reply_message(event.reply_token,TextSendMessage(text=msg))


app.run()

TypeError: ignored

In [30]:
import OPDAid3_SN
OPDAid3_SN.myLineBotApi
OPDAid3_SN.myWebhookHandler

'17a979c412f4bb0adb0014f3fcc76708'

In [33]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, abort
from datetime import date
from linebot import (LineBotApi, WebhookHandler)
from linebot.exceptions import (InvalidSignatureError)
from linebot.models import (MessageEvent, TextMessage, TextSendMessage)
from bs4 import BeautifulSoup
import requests, json
import OPDAid3_SN
today=date.today()
app=Flask(__name__)

line_bot_api=LineBotApi(OPDAid3_SN.myLineBotApi)
handler=WebhookHandler(OPDAid3_SN.myWebhookHandler)

run_with_ngrok(app)

@app.route('/')
def index():
  return '三總北投門診助手3.0 '+str(today)

@app.route('/callback',methods=['POST'])
def callback():
  signature=request.headers['X-Line-Signature']
  body=request.get_data(as_text=True)
  try:
    handler.handle(body,signature)
  except InvalidSignatureError:
    abort(400)
  return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
  from bs4 import BeautifulSoup
  import requests
  msg=event.message.text
  if msg=='醫師查詢':
    target_url='https://wwwv.tsgh.ndmctsgh.edu.tw/Doclist/194/30000/25075'
    responses=requests.get(target_url)
    soup=BeautifulSoup(responses.text,"html.parser")
    myDivs=soup.find_all('div',{'class':'column column-block lg-txt'})
    myResult=''
    for myDiv in myDivs:
      myLink=myDiv.find('a')
      myResult += str(myLink.contents[0])
      myResult += '\n'
      myResult += f"https://wwwv.tsgh.ndmctsgh.edu.tw/{myLink.get('href')}"
      myResult += '\n\n' 
    if event.source.user_id !='Udeadbeefdeadbeefdeadbeefdeadbeef':  
      line_bot_api.reply_message(event.reply_token,TextSendMessage(text=myResult))
  if msg=='門診時間':
    target_url='https://wwwv.tsgh.ndmctsgh.edu.tw/unit/30000/28387'
    responses=requests.get(target_url)
    soup=BeautifulSoup(responses.text,"html.parser")
    門診時刻表=soup.select_one('div.systems-information')
    每月時刻表=門診時刻表.find_all('a')
    myResult=''
    for tag in 每月時刻表:
      myLink=str(tag.get('href'))
      myLink=f"https://wwwv.tsgh.ndmctsgh.edu.tw/{myLink}"
      myResult +=tag.contents[0]
      myResult +='\n'
      myResult += myLink
      myResult +='\n'
    if event.source.user_id !='Udeadbeefdeadbeefdeadbeefdeadbeef':
      line_bot_api.reply_message(event.reply_token,TextSendMessage(text=myResult))

  if msg=='掛號查詢': 
    myResult='''
新民路急診部現場掛號：
  台北市北投區新民路60號
中和街門診部現場掛號：
  台北市北投區中和街250號
網路掛號：http://818h.mnd.gov.tw/Tre/start.asp 
人工掛號：02-28932743 或 02-28935869
    '''
    if event.source.user_id !='Udeadbeefdeadbeefdeadbeefdeadbeef':
      line_bot_api.reply_message(event.reply_token,TextSendMessage(text=myResult))

  if msg=='費用申請':
    target_url='https://wwwv.tsgh.ndmctsgh.edu.tw/unit/30000/16048'
    response=requests.get(target_url)
    soup=BeautifulSoup(response.text, 'html.parser')
    myDiv=soup.select_one('div.systems-information')
    收費標準s=myDiv.find_all('a')
    myResult=''
    for i, tag in enumerate(收費標準s):
      myResult+='\n {}.{}。詳情如下: \n {}'.format(i,tag.contents[0],tag.get('href'))
    if event.source.user_id !='Udeadbeefdeadbeefdeadbeefdeadbeef':
      line_bot_api.reply_message(event.reply_token,TextSendMessage(text=myResult))

  if msg=='重大公告':
    target_url='https://wwwv.tsgh.ndmctsgh.edu.tw/news/194/30000/24181'
    responses=requests.get(target_url)
    soup=BeautifulSoup(responses.text,"html.parser")
    myNews=soup.find_all('a',{'id':'clickElement'})
    myResult=''
    for i, news in enumerate(myNews):
      myResult +='第{}則.{}...'.format(i,str(news.contents[0]).replace('\n',''))
      myResult +='\n'
      myResult +='https://wwwv.tsgh.ndmctsgh.edu.tw'+news.get('href')
      myResult +='\n'
    if event.source.user_id !='Udeadbeefdeadbeefdeadbeefdeadbeef':
      line_bot_api.reply_message(event.reply_token,TextSendMessage(text=myResult))

  else:
    line_bot_api.reply_message(event.reply_token,TextSendMessage(text=msg))

'''
@handler.add(MessageEvent, message=TextMessage)
def echo(event):
  if event.source.user_id !='Udeadbeefdeadbeefdeadbeefdeadbeef':
    line_bot_api.reply_message(
        event.replay_token,
        TextSendMessage(text=event.message.text)
    )
'''
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e7bd-34-125-95-239.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [22/Aug/2021 06:51:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Aug/2021 06:51:04] "GET /favicon.ico HTTP/1.1" 404 -
[2021-08-22 06:51:32,957] ERROR in app: Exception on /callback [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.e

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, abort
from datetime import date
from linebot import (LineBotApi, WebhookHandler)
from linebot.exceptions import (InvalidSignatureError)
from linebot.models import (MessageEvent, TextMessage, TextSendMessage)
from bs4 import BeautifulSoup
import requests, json
import OPDAid3_SN
today=date.today()
app=Flask(__name__)

line_bot_api=LineBotApi(OPDAid3_SN.myLineBotApi)
handler=WebhookHandler(OPDAid3_SN.myWebhookHandler)

run_with_ngrok(app)

@app.route('/')
def index():
  return '三總北投門診助手3.0 '+str(today)

@app.route('/callback',methods=['POST'])
def callback():
  signature=request.headers['X-Line-Signature']
  body=request.get_data(as_text=True)
  app.logger.info("Request body:"+ body)

  print(body)


  try:
    handler.handle(body,signature)
  except InvalidSignatureError:
    abort(400)
  return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
  from bs4 import BeautifulSoup
  import requests
  msg=event.message.text
  if msg=='醫師查詢':
    target_url='https://wwwv.tsgh.ndmctsgh.edu.tw/Doclist/194/30000/25075'
    responses=requests.get(target_url)
    soup=BeautifulSoup(responses.text,"html.parser")
    myDivs=soup.find_all('div',{'class':'column column-block lg-txt'})
    myResult=''
    for myDiv in myDivs:
      myLink=myDiv.find('a')
      myResult += str(myLink.contents[0])
      myResult += '\n'
      myResult += f"https://wwwv.tsgh.ndmctsgh.edu.tw/{myLink.get('href')}"
      myResult += '\n\n' 
    
    line_bot_api.reply_message(event.reply_token,TextSendMessage(text=myResult))
  if msg=='門診時間':
    target_url='https://wwwv.tsgh.ndmctsgh.edu.tw/unit/30000/28387'
    responses=requests.get(target_url)
    soup=BeautifulSoup(responses.text,"html.parser")
    門診時刻表=soup.select_one('div.systems-information')
    每月時刻表=門診時刻表.find_all('a')
    myResult=''
    for tag in 每月時刻表:
      myLink=str(tag.get('href'))
      myLink=f"https://wwwv.tsgh.ndmctsgh.edu.tw/{myLink}"
      myResult +=tag.contents[0]
      myResult +='\n'
      myResult += myLink
      myResult +='\n'
    if event.source.user_id !='Udeadbeefdeadbeefdeadbeefdeadbeef':
      line_bot_api.reply_message(event.reply_token,TextSendMessage(text=myResult))

  if msg=='掛號查詢': 
    myResult='''
新民路急診部現場掛號：
  台北市北投區新民路60號
中和街門診部現場掛號：
  台北市北投區中和街250號
網路掛號：http://818h.mnd.gov.tw/Tre/start.asp 
人工掛號：02-28932743 或 02-28935869
    '''
    if event.source.user_id !='Udeadbeefdeadbeefdeadbeefdeadbeef':
      line_bot_api.reply_message(event.reply_token,TextSendMessage(text=myResult))

  if msg=='費用申請':
    target_url='https://wwwv.tsgh.ndmctsgh.edu.tw/unit/30000/16048'
    response=requests.get(target_url)
    soup=BeautifulSoup(response.text, 'html.parser')
    myDiv=soup.select_one('div.systems-information')
    收費標準s=myDiv.find_all('a')
    myResult=''
    for i, tag in enumerate(收費標準s):
      myResult+='\n {}.{}。詳情如下: \n {}'.format(i,tag.contents[0],tag.get('href'))
    if event.source.user_id !='Udeadbeefdeadbeefdeadbeefdeadbeef':
      line_bot_api.reply_message(event.reply_token,TextSendMessage(text=myResult))

  if msg=='重大公告':
    target_url='https://wwwv.tsgh.ndmctsgh.edu.tw/news/194/30000/24181'
    responses=requests.get(target_url)
    soup=BeautifulSoup(responses.text,"html.parser")
    myNews=soup.find_all('a',{'id':'clickElement'})
    myResult=''
    for i, news in enumerate(myNews):
      myResult +='第{}則.{}...'.format(i,str(news.contents[0]).replace('\n',''))
      myResult +='\n'
      myResult +='https://wwwv.tsgh.ndmctsgh.edu.tw'+news.get('href')
      myResult +='\n'
    if event.source.user_id !='Udeadbeefdeadbeefdeadbeefdeadbeef':
      line_bot_api.reply_message(event.reply_token,TextSendMessage(text=myResult))

  else:
    line_bot_api.reply_message(event.reply_token,TextSendMessage(text=msg))

'''
@handler.add(MessageEvent, message=TextMessage)
def echo(event):
  if event.source.user_id !='Udeadbeefdeadbeefdeadbeefdeadbeef':
    line_bot_api.reply_message(
        event.replay_token,
        TextSendMessage(text=event.message.text)
    )
'''
app.run()

In [35]:
import OPDAid3_SN
OPDAid3_SN.myUserID

'U2082e5cb4d9ebfc6697c8d59e56c1d8b'